# Jacobian Pseudo Inverse Controller
ToDo: Add theory of operation

## Run In Simulation

In [9]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
from FR3Py.envs.pybullet import FR3Sim

robot = FR3Sim(render_mode='human', mode='velocity', Ts=1/1000)
robot.reset()
controller = WaypointController(kp=2)
while True:
    q, dq = robot.get_state()
    dq_cmd = controller.compute(q, dq)
    robot.step(dq_cmd)
    time.sleep(1/1000)

pybullet build time: May 20 2022 19:44:17


: 

: 

## Run With Real Robot

In [ ]:
from FR3Py.interfaces import FR3Real
robot = FR3Real()

In [ ]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
controller = WaypointController(kp=1.5)
# Read the initila pose of the robot
time.sleep(1)
state = robot.get_state()

q, dq = state['q'], state['dq']
p0 = controller.robot.getInfo(q,dq)['P_EE']
R0 = controller.robot.getInfo(q,dq)['R_EE']
T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
start_time = time.time()
while True:
    time.sleep(0.01)
    t = time.time()-start_time
    x = 0.1*np.sin(t)
    T= T0@np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
    state = robot.get_state()
    q, dq = state['q'], state['dq']
    cmd = controller.compute(q,dq, T_cmd=T)
    robot.send_joint_command(cmd[0:7])

# TF Pose Capture Node

Run the following cell to repeatedly read the pose of the wall published as a TF message and set it to a global valirable pose:

In [1]:
import rospy
import tf
import time
import threading
global pose
from scipy.spatial.transform import Rotation
import numpy as np
import pickle
from copy import deepcopy

rospy.init_node("tf_listener_node")

In [2]:
def tf_listener_thread():

    def get_T(p, R):
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = p

        return T

    global pose
    global board_pos
    global board_ori

    camera_to_base_transform = np.load("/home/franka-ws/manipulation/data/camera_to_grey_base_transform.npy")

    listener = tf.TransformListener()

    # the frame where the data originated
    target_frame = "/camera_color_optical_frame"

    # the frame where the data should be transformed
    source_frame = "/wall"

    listener.waitForTransform(target_frame, source_frame, rospy.Time(), rospy.Duration(4.0))

    # Main loop to continuously update the pose
    while not rospy.is_shutdown():
        try:
            (trans, quat) = listener.lookupTransform(target_frame, source_frame, rospy.Time(0))
            
            pos = np.array(trans)
            Rot = Rotation.from_quat(quat).as_matrix()

            T = camera_to_base_transform @ get_T(pos, Rot)
            board_pos = T[:3, 3]
            board_rot = T[:3, :3]
            board_ori = Rotation.from_matrix(board_rot).as_quat()

            rospy.sleep(0.001)  # Adjust the sleep duration as needed
        except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
            rospy.logwarn("Error occurred while retrieving TF transform.")

# Initialize the global pose variable
pose = None

# Create and start the thread
tf_thread = threading.Thread(target=tf_listener_thread)
tf_thread.start()

In [6]:
dataset = []
initial_time = time.time()

for i in range(int(1200)):
    dataset.append(
        {
            "time": time.time() - initial_time,
            "board_pos": deepcopy(board_pos),
            "board_ori": deepcopy(board_ori),
        }
    )

    time.sleep(1/60)

KeyboardInterrupt: 

In [ ]:
 with open("../data/moving_board_data.pickle", "wb") as handle:
        pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)